In [11]:
import pandas as pd
import spacy
from spacy.training import Example
csv_file  = 'sms.csv'
print(spacy.__version__)


def preprocess_data(csv_file):
    df = pd.read_csv(csv_file)
    training_data = []

    for _, row in df.iterrows(): 
        text = row['sms']
        entities = [] 

        # Extract entities without overlaps
        start_amount = text.find(str(row['amount']))
        end_amount = start_amount + len(str(row['amount']))
        entities.append((start_amount, end_amount, 'amount'))

        start_fees = text.find(str(row['fees']))
        end_fees = start_fees + len(str(row['fees']))
        entities.append((start_fees, end_fees, 'fees'))

        start_date = text.find(row['date_time']) 
        end_date = start_date + len(row['date_time'])
        entities.append((start_date, end_date, 'date_time')) 
        
        start_transaction_type = text.find(row['transaction_type'])
        end_transaction_type = start_transaction_type + len(row['transaction_type'])
        entities.append((start_transaction_type, end_transaction_type, 'transaction_type'))
        
        start_receiver_sender = text.find(row['receiver_sender'])
        end_receiver_sender = start_receiver_sender + len(row['receiver_sender'])
        entities.append((start_receiver_sender, end_receiver_sender, 'receiver_sender'))

        # Adjust entities to resolve overlaps
        # For example, prioritize longer entities over shorter ones
        adjusted_entities = resolve_overlaps(entities)

        training_data.append((text, {"entities": adjusted_entities}))
        biluo_tags = spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)
        print(biluo_tags)

    return training_data

def resolve_overlaps(entities):
    adjusted_entities = []
    for start, end, label in entities:
        # Check if the current entity overlaps with any previous entity
        is_overlapping = any(start < prev_end and end > prev_start for prev_start, prev_end, _ in adjusted_entities)
        if not is_overlapping:
            # If no overlap, add the entity as is
            adjusted_entities.append((start, end, label))
        else:
            # Resolve overlap based on prioritization or splitting
            # For example, you can prioritize longer entities
            prev_entity_length = sum(prev_end - prev_start for prev_start, prev_end, _ in adjusted_entities)
            current_entity_length = end - start
            if current_entity_length > prev_entity_length:
                # Prioritize the current entity
                adjusted_entities = [(s, e, l) for s, e, l in adjusted_entities if not (start < e and end > s)]
                adjusted_entities.append((start, end, label))
    
    return adjusted_entities


# def preprocess_data(csv_file):
#     df = pd.read_csv(csv_file)
#     training_data = []

#     for _, row in df.iterrows(): 
#         text = row['sms']
#         entities = [] 

#         start_amount = text.find(str(row['amount']))
#         end_amount = start_amount + len(str(row['amount']))
#         entities.append((start_amount, end_amount, 'amount'))

#         start_fees = text.find(str(row['fees']))
#         end_fees = start_fees + len(str(row['fees']))
#         entities.append((start_fees, end_fees, 'fees'))

#         start_date = text.find(row['date_time']) 
#         end_date = start_date + len(row['date_time'])
#         entities.append((start_date, end_date, 'date_time')) 
        
#         start_transaction_type = text.find(row['transaction_type'])
#         end_transaction_type = start_transaction_type + len(row['transaction_type'])
#         entities.append((start_transaction_type, end_transaction_type, 'transaction_type'))
        
#         start_receiver_sender = text.find(row['receiver_sender'])
#         end_receiver_sender = start_receiver_sender + len(row['receiver_sender'])
#         entities.append((start_receiver_sender, end_receiver_sender, 'receiver_sender'))

#         training_data.append((text, {"entities": entities}))
#     return training_data


# def preprocess_data(csv_file):
#     df = pd.read_csv(csv_file)
#     training_data = []

#     for _, row in df.iterrows(): 
#         text = row['sms']
#         entities = [] 

#         start_amount = text.find(str(row['amount']))
#         end_amount = start_amount + len(str(row['amount']))
#         entities.append((start_amount, end_amount, 'amount'))

#         start_fees = text.find(str(row['fees']))
#         end_fees = start_fees + len(str(row['fees']))
#         entities.append((start_fees, end_fees, 'fees'))

#         start_date = text.find(row['date_time']) 
#         end_date = start_date + len(row['date_time'])
#         entities.append((start_date, end_date, 'date_time')) 
        
#         start_transaction_type = text.find(row['transaction_type'])
#         end_transaction_type = start_transaction_type + len(row['transaction_type'])
#         entities.append((start_transaction_type, end_transaction_type, 'transaction_type'))
        

#         # entities.append((text.find(str(row['amount'])), text.find(str(row['amount'])) + len(str(row['amount'])), 'amount'))
#         # entities.append((text.find(row['transaction_type']), text.find(row['transaction_type']) + len(row['transaction_type']), 'transaction_type'))
#         entities.append((text.find(row['receiver_sender']), text.find(row['receiver_sender']) + len(row['receiver_sender']), 'receiver_sender'))
#         # entities.append((text.find(str(row['fees'])), text.find(str(row['fees'])) + len(str(row['fees'])), 'fees'))
#         # entities.append((text.find(row['acccount']), text.find(row['acccount']) + len(row['acccount']), 'account'))
#         # ... add more entities similarly (fees, account, receiver_sender)

#         # training_data.append((text, {"entities": entities}))
#         training_data.append((text, {"spans": entities}))
#     return training_data



3.7.4


In [12]:
nlp = spacy.blank("en")  # Create a blank English model
# ner = nlp.add_pipe("ner")  # Add the NER component

spancat = nlp.add_pipe("spancat")


# Add your custom labels
spancat.add_label("date_time")
spancat.add_label("amount")
spancat.add_label("transaction_type")
spancat.add_label("receiver_sender")
spancat.add_label("fees")
# snapcat.add_label("account")

optimizer = nlp.begin_training()
training_data = preprocess_data(csv_file) 

for i in range(20):  # Example: Train for 20 iterations
    examples = []
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        print(example)
        examples.append(example)
    nlp.update(examples, sgd=optimizer)
nlp.to_disk("sms_parser_model") 

ValueError: [E103] Trying to set conflicting doc.ents: '(18, 23, 'amount')' and '(19, 20, 'fees')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.

In [9]:
loaded_model = spacy.load("sms_parser_model")

new_sms = "TxId: 13113290018. Your payment of 1,300 RWF to EMMANUELIE  UWIMANA 515867 has been completed at 2024-03-13 17:56:32. Your new balance: 2,002 RWF. Fee was 0 RWF"  
carrier = "M-Money" # You'll provide this 
doc = loaded_model(new_sms)

print(doc)
extracted_data = {
    "carrier": carrier    
}

for ent in doc.ents:
    extracted_data[ent.label_] = ent.text 

print(extracted_data) 

TxId: 13113290018. Your payment of 1,300 RWF to EMMANUELIE  UWIMANA 515867 has been completed at 2024-03-13 17:56:32. Your new balance: 2,002 RWF. Fee was 0 RWF
{'carrier': 'M-Money'}
